In [1]:
import numpy as np

On veut calculer le produit de deux matrics $A$ et $B$ noté $C = AB$ de taille $n \times n$.

# Methode de l'exercice 21.

On a $C_{i,j} = \sum_{k=1}^n A_{i,k} B_{k,j}$.


In [14]:
def classic_matrix_mult(A,B):
    n = A.shape[0]
    C = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            for k in range(n):
                C[i,j] += A[i,k]*B[k,j]

    return C

# Méthode de l'exercice 41

Divide and conquer en 8 sous matrices

In [15]:
def divide_and_conquer_matrix_mult(A,B):
    n = A.shape[0]
    if n == 1:
        return np.array([[A[0,0]*B[0,0]]])
    else:
        m = n//2
        A11 = A[:m,:m]
        A12 = A[:m,m:]
        A21 = A[m:,:m]
        A22 = A[m:,m:]
        B11 = B[:m,:m]
        B12 = B[:m,m:]
        B21 = B[m:,:m]
        B22 = B[m:,m:]
        C11 = divide_and_conquer_matrix_mult(A11,B11) + divide_and_conquer_matrix_mult(A12,B21)
        C12 = divide_and_conquer_matrix_mult(A11,B12) + divide_and_conquer_matrix_mult(A12,B22)
        C21 = divide_and_conquer_matrix_mult(A21,B11) + divide_and_conquer_matrix_mult(A22,B21)
        C22 = divide_and_conquer_matrix_mult(A21,B12) + divide_and_conquer_matrix_mult(A22,B22)
        C = np.zeros((n,n))
        C[:m,:m] = C11
        C[:m,m:] = C12
        C[m:,:m] = C21
        C[m:,m:] = C22
        return C

# Exercice 2

In [16]:
def matrix_plus(A,B):
    n = A.shape[0]
    C = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            C[i,j] = A[i,j] + B[i,j]
    return C

In [17]:
def matrix_minus(A,B):
    n = A.shape[0]
    C = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            C[i,j] = A[i,j] - B[i,j]
    return C

In [18]:
def mult_strassen(A,B):
    n = A.shape[0]
    if n == 1:
        return np.array([[A[0,0]*B[0,0]]])
    else:
        m = n//2
        A11 = A[:m,:m]
        A12 = A[:m,m:]
        A21 = A[m:,:m]
        A22 = A[m:,m:]
        B11 = B[:m,:m]
        B12 = B[:m,m:]
        B21 = B[m:,:m]
        B22 = B[m:,m:]
        P1 = mult_strassen(matrix_plus(A11,A22),matrix_plus(B11,B22))
        P2 = mult_strassen(matrix_plus(A21,A22),B11)
        P3 = mult_strassen(A11,matrix_minus(B12,B22))
        P4 = mult_strassen(A22,matrix_minus(B21,B11))
        P5 = mult_strassen(matrix_plus(A11,A12),B22)
        P6 = mult_strassen(matrix_minus(A21,A11),matrix_plus(B12,B11))
        P7 = mult_strassen(matrix_minus(A12,A22),matrix_plus(B21,B22))
        C11 = matrix_plus(matrix_minus(matrix_plus(P1,P4),P5),P7)
        C12 = matrix_plus(P3,P5)
        C21 = matrix_plus(P2,P4)
        C22 = matrix_plus(matrix_minus(matrix_plus(P1,P3),P2),P6)
        C = np.zeros((n,n))
        C[:m,:m] = C11
        C[:m,m:] = C12
        C[m:,:m] = C21
        C[m:,m:] = C22
        return C

On teste que les algos renvoient la même chose sur quelques exemples

In [19]:
A = np.array([[1,2],[3,4]])
B = np.array([[5,6],[7,8]])
print(classic_matrix_mult(A,B))
print(divide_and_conquer_matrix_mult(A,B))
print(mult_strassen(A,B))

[[19. 22.]
 [43. 50.]]
[[19. 22.]
 [43. 50.]]
[[19. 22.]
 [43. 50.]]


In [26]:
A = np.random.randint(10,size=(8,8))
B = np.random.randint(10,size=(8,8))
print((classic_matrix_mult(A,B) == divide_and_conquer_matrix_mult(A,B)).all())
print((classic_matrix_mult(A,B) == mult_strassen(A,B)).all())

True
True


# Exercice 3 

Complexité pire cas des versions naïves : $\Theta(n^3)$

Complexité pire cas de l'algorithme de Strassen :

$T(n) = 7T(n/2) + \Theta(n^2)$

On a $log_b(a) = log_2(7) > 2$ donc $T(n) = \Theta(n^{log_2(7)})$

In [31]:
import time

n_trys = 1000

total_time_classic = 0
total_time_divide = 0
total_time_strassen = 0
total_time_numpy = 0

for i in range(n_trys):
    A = np.random.randint(10,size=(8,8))
    B = np.random.randint(10,size=(8,8))
    t0 = time.time()
    classic_matrix_mult(A,B)
    t1 = time.time()
    divide_and_conquer_matrix_mult(A,B)
    t2 = time.time()
    mult_strassen(A,B)
    t3 = time.time()
    np.dot(A,B)
    t4 = time.time()
    total_time_classic += t1-t0
    total_time_divide += t2-t1
    total_time_strassen += t3-t2
    total_time_numpy += t4-t3


print(total_time_classic)
print(total_time_divide)
print(total_time_strassen)
print(total_time_numpy)

0.23881745338439941
0.9493660926818848
2.001481294631958
0.0050201416015625


On constate que l'algo de Strassen est le plus long. Explication : la constante associée à l'algo de Strassen est grande, donc sur les petites matrices il sera plus long. Pour que l'algo de Strassen devienne plus efficace, il faut tester sur des matrices bien plus grandes.

La fonction $np.dot$ de la bibliothèque numpy est encore plus rapide car ils utilisent des fonctions codées en C et non en python pour calculer le résultat du produit (et le C est bien plus efficace que le python).